In [18]:
df = spark.read.format('com.databricks.spark.csv').\
                               options(header='true', \
                               inferschema='true') \
                .load("/home/bigdata/unswnb15_old.csv",header=True);
#df.show(5,True)

In [8]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['dur',
                                               'sbytes',
                                               'dbytes',
                                               'sttl',
                                               'dttl',
                                               'sloss',
                                               'dloss',
                                               'sload',
                                               'dload',
                                               'spkts',
                                               'tcprtt',
                                               'synack',
                                               'is_sm_ips_ports',
                                               'dpkts',
                                               'ct_dst_sport_ltm',
                                               'ct_dst_src_ltm',
                                               'label'], outputCol = 'features')
final_data = vectorAssembler.setHandleInvalid("keep").transform(df)
final_data = v_unsw_data.select(['features', 'label'])
final_data.show(3)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.022316,3598.0,...|    0|
|[1.064936,1580.0,...|    0|
|[0.072091,2958.0,...|    0|
+--------------------+-----+
only showing top 3 rows



Scale the data 

In [12]:
from pyspark.ml.clustering import KMeans
# Loads data.
#dataset = spark.read.csv('/home/bigdata/Downloads/UNSW-NB15(1).csv',header=True,inferSchema=True)

In [4]:
from pyspark.ml.feature import StandardScaler

In [5]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [9]:
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(final_data)

In [10]:
# Normalize each feature to have unit standard deviation.
final_data = scalerModel.transform(final_data)

In [13]:
# Trains a k-means model.
kmeans = KMeans(featuresCol='scaledFeatures',k=3)
model = kmeans.fit(final_data)

In [14]:
# Evaluate clustering by computing Within Set Sum of Squared Errors.
#wssse = model.computeCost(final_data)
wssse = model.summary.trainingCost
print("Within Set Sum of Squared Errors = " + str(wssse))

Within Set Sum of Squared Errors = 8350035.354570394


In [15]:
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[0.01732726 0.08893118 0.07743694 0.7746528  0.93394914 0.29681835
 0.14847069 0.05778186 0.70245686 0.36971625 0.02060979 0.02947867
 0.05015516 0.22039721 0.91441407 0.94325221 0.17727475]
[3.95213248e-02 1.07240735e-01 2.23398854e-02 5.54222336e+00
 3.84517569e+00 1.56492762e-01 3.92853206e-02 1.59784205e+00
 4.65078234e-03 1.30338539e-01 2.69264872e+00 2.52682143e+00
 4.20932843e-02 5.56037103e-02 1.42796568e+00 1.24770868e+00
 2.53813847e-01]
[3.97524980e-01 6.41434540e-01 4.72714461e+00 7.80263198e-01
 9.74350383e-01 1.41711845e+00 4.81788648e+00 1.19338693e-03
 5.56911181e-01 4.40880583e+00 3.97697834e-02 5.00048460e-02
 5.55263578e-02 4.86730904e+00 2.13438340e+00 1.80666607e+00
 1.96335340e-01]


In [16]:
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         2|
|         0|
|         0|
|         2|
|         0|
|         0|
|         0|
+----------+
only showing top 20 rows

